In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
252,AFG,Asia,Afghanistan,2020-09-08,38494.0,96.0,42.571,1415.0,3.0,1.286,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
436,ALB,Europe,Albania,2020-09-08,10406.0,151.0,127.571,319.0,3.0,5.000,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
689,DZA,Africa,Algeria,2020-09-08,46653.0,289.0,308.429,1562.0,6.0,7.429,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-07,France,332047.0,0.0,30731.0,0.0,0.0,0.0
0,2020-09-07,China,90086.0,0.0,4736.0,0.0,0.0,0.0
0,2020-09-07,Italy,279136.0,0.0,35575.0,0.0,0.0,0.0
0,2020-09-07,Spain,500416.0,0.0,29426.0,0.0,0.0,0.0
0,2020-09-07,United States,6363730.0,0.0,190102.0,0.0,0.0,0.0
0,2020-09-07,World,27679267.0,0.0,895813.0,0.0,0.0,0.0
0,2020-09-07,United Kingdom,348702.0,0.0,41596.0,0.0,0.0,0.0
0,2020-09-07,Germany,251768.0,0.0,9332.0,0.0,0.0,0.0
0,2020-09-07,Iran,390618.0,0.0,22524.0,0.0,0.0,0.0
0,2020-09-07,Turkey,281157.0,0.0,6718.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-08,France,332047.0,328980.0,30731.0,30726.0,3067.0,5.0
0,2020-09-08,China,90086.0,90079.0,4736.0,4732.0,7.0,4.0
0,2020-09-08,Italy,279136.0,278784.0,35575.0,35553.0,352.0,22.0
0,2020-09-07,Spain,500416.0,525549.0,29426.0,29516.0,-25133.0,-90.0
0,2020-09-08,United States,6363730.0,6300671.0,190102.0,189208.0,63059.0,894.0
0,2020-09-08,World,27679267.0,27366648.0,895813.0,893084.0,312619.0,2729.0
0,2020-09-08,United Kingdom,348702.0,350100.0,41596.0,41554.0,-1398.0,42.0
0,2020-09-08,Germany,251768.0,252298.0,9332.0,9329.0,-530.0,3.0
0,2020-09-08,Iran,390618.0,388810.0,22524.0,22410.0,1808.0,114.0
0,2020-09-08,Turkey,281157.0,281509.0,6718.0,6730.0,-352.0,-12.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")